In [ ]:
import pandas as pd
import os
import glob

# How Big is Canada, Really?

Canada is the second largest country in the world by area, but most of it is virtually uninhabited. In 2016, 66% of the Canadian population lived within 100km of the US border [source](https://www150.statcan.gc.ca/n1/daily-quotidien/170208/dq170208a-eng.htm).
I've been curious for a long while now, how big is Canada really, if we only include area that has a certain population density or higher?

A naive approach would to go to every country's own statistics/census website, wrestle with their chosen data accessing and formatting patterns.
A slightly smarter approach is to let someone else do it for us, and use their results.
Luckily, the [Socioeconomic Data and Applications Center (sedac)](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4) has done this since 1995.

I originally downloaded their Population Density dataset v4.11, which contains ASCII and TIFF files of pixel-level (up to a resolution of 30 arc-second) data.
Unfortunately, I couldn't think of a convenient way of figuring out which pixels belong to each country.
A much easier approach is to use the Administrative Unit Center Points with Population Estimates dataset, which gives, per administrative unit in a given country, both its population and area, which makes getting the density easy as pie.
I'm not sure if there's some resolution lost by solving this way, since a hypothetically gigantic admin unit could have everyone concentrated in a single square kilometer, and we'd never know.
Something to look out for when we start wrangling the data.

## Step 1: Download Data

I went over the the link above (or [here](https://sedac.ciesin.columbia.edu/data/collection/gpw-v4), same link), create an account, selected Global/Regional as the Geography, Comma Separated Value as the file format, and then made a decision: either

1. Tick the Global box, which contains a single CSV for the rest of the world minus the US, then four separate CSV for the latter, or
2. Tick the rest of the boxes, which will split the CSV files for the rest of the world into continents (with four separate CSV files for the US again)

I figure this hinges on how much RAM these files would occupy, and whether I wanted to parallelize processing.

The single global CSV is 3.1GB, while the CSV files for subsections of the US are up to 2.7GB.
The files for continents are all under 1.5GB. According to [Jeff](https://stackoverflow.com/questions/25962114/how-do-i-read-a-large-csv-file-with-pandas), it takes about double the size of the file in RAM to open it up; since I won't be doing much more than removing unecessary columns, it's relatively affordable.

Since I'm lazy and have my fingers crossed this'll run relatively quickly given how simple what I'm doing will be, I won't bother parallelizing either.

So Global download only it is! I also downloaded the documentation, since it describes the column titles used below.

## Step 2: Load, Trim, Permute, and Save

Next step is pretty straightforward; load each CSV, remove all columns but the country, density, and area (I'll keep population too for futzing around later), and save it all to a CSV for probably faster loading if I ever want to open these again.

In [ ]:
curr_dir = os.getcwd()
files = glob.glob(curr_dir + '/data/unprocessed/*.csv')
combined_csv = pd.DataFrame()

for file in files:
    df = pd.read_csv(file)
    df = df[['COUNTRYNM', 'LAND_A_KM', 'UN_2020_E', 'UN_2020_DS']]
    combined_csv = pd.concat([combined_csv, df], ignore_index=True)

combined_csv.to_csv(curr_dir + '/data/processed/data.csv', index=False) # creative naming, I know

That trims our CSV file down to a much more manageable 700MB.
Let's load it back up and make sure everything is in the right ballpark.

In [ ]:
processed_dir = curr_dir + '/data/processed/data.csv'
world = pd.read_csv(processed_dir)

canada = world.loc[world['COUNTRYNM'] == 'Canada']
print("Some basic stats to make sure we're in the right ballbark:\n" + canada[['LAND_A_KM', 'UN_2020_E']].sum().to_string())

In [ ]:
world.nlargest(10, 'LAND_A_KM')

Population of 37 million and land area of 9 million? Seems about right.

Unfortunately, those administrative units are absolutely massive.
The Canadian one isn't so problematic, since the population is zero, and there's no way to distribute zero people that will influence the density of any square kilimeter of land within it.

The others are more problematic.
The fourth largest, for example, contains 1.6 million people, but with an average density of 3.3 people per square kilometer, if our threshold were set to, for example, the average population density of the world (about [50/sqkm](https://en.wikipedia.org/wiki/Population_density)), then the entire area would be ignored.
Not much I can think about off the top of my head to fix this, so we'll just have to keep it in mind when looking at the results.

## Step 3: How Dense is Dense Enough?

What density is reasonable to consider an administrative unit sufficiently populated to garner interest?
There's a few approaches we can take.

We can trim all units with a density below the global average of 50 people per square kilometer above; this'll hurt Canada's area quite a bit, but that's what we're here for.

We can also trim all units with a density below the average density in north-american farmland. [Statistics Canada](https://www150.statcan.gc.ca/n1/pub/95-640-x/2011001/p1/p1-01-eng.htm) lists 160 155 748 acres of farmland in Canada in 2011, or 648 127sqkm. With about [241 500](https://agriculture.canada.ca/en/sector/overview) jobs in primary agriculture, that comes out to 2.68 people per square kilometer, significantly lower than the global average, and a generous threshold to let Canada keep some area in the upcoming smackdown.

We can look at the least dense US state and use that as our threshold. Why? No clue, seems like fun. [Wikipedia](https://en.wikipedia.org/wiki/List_of_states_and_territories_of_the_United_States_by_population_density) to the rescue, which reports Alaska as our lucky winner. I was originally going to exclude Alaska, since I figured it would be ridiculously low, but considering the second lowest, Wyoming, has a density of 2.3 people per sqkm, which is about the same as for the average Canadian farm, looks like we're using our friend up north and being even more gentle with Canada's new area.

## Step 4: Shrink Canada